<img src="./img/oscon.png" width="400">

# Pre-trained word vectors

Here, we show how to take [GloVe](http://nlp.stanford.edu/projects/glove/) word vectors trained based on Common Crawl web data, and incorporate them into a our text classification.

Start by setting the path to which you downloaded the pre-trained vectors.


In [1]:
import sys,os
DATA_DIR = os.path.join(os.environ["HOME"], "data") if not 'win32' in sys.platform else "c:\\tmp\\data"
path_to_glove = os.path.join(DATA_DIR,"glove.840B.300d.zip")

In [2]:
import numpy as np
digit_to_word_map = {1:"One",2:"Two", 3:"Three", 4:"Four", 5:"Five",
                     6:"Six",7:"Seven",8:"Eight",9:"Nine"}
digit_to_word_map[0]="PAD_TOKEN"
times_steps = 6
even_sentences = []
odd_sentences = []
seqlens = []
for i in range(10000):
    rand_seq_len = np.random.choice(range(3,times_steps+1))
    seqlens.append(rand_seq_len)
    rand_odd_ints = np.random.choice(range(1,10,2),
                                     rand_seq_len)
    rand_even_ints = np.random.choice(range(2,10,2),
                                      rand_seq_len)

    #Padding
    if rand_seq_len<times_steps:
        rand_odd_ints = np.append(rand_odd_ints,
                                  [0]*(times_steps-rand_seq_len))
        rand_even_ints = np.append(rand_even_ints,
                                   [0]*(times_steps-rand_seq_len))

    even_sentences+=[" ".join([digit_to_word_map[r] for
                               r in rand_even_ints])]
    odd_sentences+=[" ".join([digit_to_word_map[r] for
                              r in rand_odd_ints])] 

data = even_sentences+odd_sentences
seqlens = seqlens + seqlens
#Map from words to indices
word2index_map ={}
index=0
for sent in data:
    for word in sent.lower().split():
        if word not in word2index_map:
            word2index_map[word] = index
            index+=1
#Inverse map    
index2word_map = dict([(index,word) for word,index in
                       word2index_map.items()])            
vocabulary_size = len(index2word_map)
labels = [1]*10000 + [0]*10000
for i in range(len(labels)):
    label = labels[i]
    one_hot_encoding = [0]*2
    one_hot_encoding[label] = 1
    labels[i] = one_hot_encoding


data_indices = list(range(len(data)))
np.random.shuffle(data_indices)
data = np.array(data)[data_indices]

labels = np.array(labels)[data_indices]
seqlens = np.array(seqlens)[data_indices]
train_x = data[:10000]
train_y = labels[:10000]
train_seqlens = seqlens[:10000]

test_x = data[10000:]
test_y = labels[10000:]
test_seqlens = seqlens[10000:]
def get_sentence_batch(batch_size,data_x,
                       data_y,data_seqlens):
    instance_indices = list(range(len(data_x)))
    np.random.shuffle(instance_indices)
    batch = instance_indices[:batch_size]
    x = [[word2index_map[word] for word in data_x[i].lower().split()]
         for i in batch]
    y = [data_y[i] for i in batch]
    seqlens = [data_seqlens[i] for i in batch]
    return x,y,seqlens

There are 2.2 million words in the vocabulary of the pre-trained GloVe embeddings we downloaded — we only take the GloVe vectors for words that appear in our own, tiny vocabulary.

In [3]:
import zipfile
def get_glove(path_to_glove,word2index_map):
        
    embedding_weights = {}
    count_all_words = 0 
    with zipfile.ZipFile(path_to_glove) as z:
        with z.open("glove.840B.300d.txt") as f:
            for line in f:
                vals = line.split()
                word = str(vals[0].decode("utf-8")) 
                if word in word2index_map:
                    print(word)
                    count_all_words+=1
                    coefs = np.asarray(vals[1:], dtype='float32')
                    coefs/=np.linalg.norm(coefs) 
                    embedding_weights[word] = coefs
                if count_all_words==len(word2index_map)-1:
                    break
    return embedding_weights 


The output of our function is a dictionary, mapping from each word to its vector.


In [4]:
word2embedding_dict = get_glove(path_to_glove,word2index_map)
print(word2embedding_dict["one"])

one
two
three
four
five
six
seven
eight
nine
[  1.17593138e-02   2.41619311e-02  -3.16584334e-02  -5.28343767e-03
   8.28812942e-02  -3.12930681e-02  -3.91418897e-02  -6.75903335e-02
  -1.73309119e-03   5.77146113e-01  -2.77764332e-02   5.19448891e-03
   1.55027825e-02  -3.10212206e-02   2.37900410e-02  -2.31984966e-02
  -8.30291770e-03   2.57777989e-01   1.14304852e-02  -3.73650864e-02
  -4.05141823e-02  -3.59558244e-03  -1.82699859e-02   3.16258147e-02
   3.49249691e-02  -1.50415087e-02  -1.67050064e-02  -4.49268371e-02
   3.48510258e-02   4.65514027e-02   1.28767211e-02   1.04152925e-01
   1.81321036e-02   4.34827730e-02   2.09269281e-02  -2.18196809e-02
   9.25960764e-03  -2.03532185e-02  -2.91182790e-02  -7.11243674e-02
   5.14338119e-03   7.40820915e-02  -1.65231936e-02  -2.20741294e-02
   2.70348303e-02  -1.21688265e-02  -5.49874231e-02  -4.02771309e-03
   1.02023799e-02  -1.84365753e-02  -3.45443822e-02   5.16534634e-02
   1.65521186e-02   1.56047801e-02   6.82993140e-03   7.10

We place these vectors in a matrix, where each row index corresponds to the word index

In [5]:
GLOVE_SIZE = 300

embedding_matrix = np.zeros((len(word2index_map),GLOVE_SIZE))

for word,index in word2index_map.items():
    if not word == "pad_token":
        word_embedding = word2embedding_dict[word]
        embedding_matrix[index,:] = word_embedding

Note that for the "pad_token" word, we set the corresponding vector to 0. The padded vectors are ignored by dynamic_rnn(), as we saw above.

Now, back to TensorFlow. First, we create the exact same placeholders we used above.

In [6]:
import tensorflow as tf
batch_size=128;num_classes = 2
tf.reset_default_graph()
sess = tf.InteractiveSession()

_inputs = tf.placeholder(tf.int32, shape=[batch_size,times_steps])
_labels = tf.placeholder(tf.float32, shape=[batch_size, num_classes])
_seqlens = tf.placeholder(tf.int32, shape=[batch_size])

Next, we create an embedding_placeholder, to which we feed the word vectors.

In [7]:
embedding_placeholder = tf.placeholder(tf.float32, [vocabulary_size,
                                                    GLOVE_SIZE])

 Our embeddings are initialized with the content of embedding_placeholder, using the assign function to assign initial values to the embeddings variable.   We set trainable=True, to tell TensorFlow we want to update the values of the word vectors, by optimizing them for the task at hand. 

In [8]:
PRE_TRAINED = True

if PRE_TRAINED:
    with tf.device('/cpu:0'):
            embeddings = tf.Variable(tf.constant(0.0, shape=[vocabulary_size, GLOVE_SIZE]),
                                     trainable=True)
            embedding_init = embeddings.assign(embedding_placeholder)
            embed = tf.nn.embedding_lookup(embeddings, _inputs)

else:
    with tf.device('/cpu:0'):
            embeddings = tf.Variable(
                tf.random_uniform([vocabulary_size,
                                   embedding_dimension],
                                  -1.0, 1.0))
            embed = tf.nn.embedding_lookup(embeddings, _inputs)

 Next, we repeat the exact same LSTM network structure we used above

In [9]:
hidden_layer_size = 32
with tf.variable_scope("lstm"):
 
    lstm_cell = tf.contrib.rnn.LSTMCell(hidden_layer_size)
    _, states = tf.nn.dynamic_rnn(lstm_cell, embed,
                                        sequence_length = _seqlens,
                                        dtype=tf.float32)
last_state = states[1]

W = tf.Variable(tf.truncated_normal([hidden_layer_size,num_classes],mean=0,stddev=.01))
b = tf.Variable(tf.truncated_normal([num_classes],mean=0,stddev=.01))

#extract the final state and use in a linear layer
final_pred = tf.matmul(last_state,W) + b
#
softmax = tf.nn.softmax_cross_entropy_with_logits(logits  = final_pred,labels = _labels)                         
cross_entropy = tf.reduce_mean(softmax)
train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(_labels,1),
                              tf.argmax(final_pred,1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction,
                                   tf.float32)))*100


 Finally, we are ready to train. We initialize the embedding_placeholder by feeding it our embedding_matrix. Importantly, we do so after calling tf.global_variables_initializer() — doing so in the reverse order, would overrun the pre-trained vectors with a default initializer!

In [10]:
sess.run(tf.global_variables_initializer())
sess.run(embedding_init, feed_dict=
         {embedding_placeholder: embedding_matrix})
for step in range(1000):
    x_batch, y_batch,seqlen_batch = get_sentence_batch(batch_size,
                                                       train_x,train_y,
                                                       train_seqlens)
    sess.run(train_step,feed_dict={_inputs:x_batch, _labels:y_batch,
                                   _seqlens:seqlen_batch})

    if step % 100 == 0:
        acc = sess.run(accuracy,feed_dict={_inputs:x_batch,
                                           _labels:y_batch,
                                           _seqlens:seqlen_batch})
        print("Accuracy at %d: %.5f" % (step, acc)) 

for test_batch in range(5):
    x_test, y_test,seqlen_test = get_sentence_batch(batch_size,
                                                    test_x,test_y,
                                                    test_seqlens)
    batch_pred,batch_acc = sess.run([tf.argmax(final_pred,1),
                                     accuracy],
                                    feed_dict={_inputs:x_test,
                                               _labels:y_test,
                                               _seqlens:seqlen_test})
    print("Test batch accuracy %d: %.5f" % (test_batch, batch_acc))   
    print("Test batch accuracy %d: %.5f" % (test_batch, batch_acc))

Accuracy at 0: 51.56250
Accuracy at 100: 48.43750
Accuracy at 200: 100.00000
Accuracy at 300: 100.00000
Accuracy at 400: 100.00000
Accuracy at 500: 100.00000
Accuracy at 600: 100.00000
Accuracy at 700: 100.00000
Accuracy at 800: 100.00000
Accuracy at 900: 100.00000
Test batch accuracy 0: 100.00000
Test batch accuracy 0: 100.00000
Test batch accuracy 1: 100.00000
Test batch accuracy 1: 100.00000
Test batch accuracy 2: 100.00000
Test batch accuracy 2: 100.00000
Test batch accuracy 3: 100.00000
Test batch accuracy 3: 100.00000
Test batch accuracy 4: 100.00000
Test batch accuracy 4: 100.00000


## Quick hands-on
* Extract the trained embedding vectors
* Choose a word (from "one" to "nine") and find its nearest neighbors -- sort all the other word vectors by their distance to the target word. Use numpy or sklearn.
* Re-run the network, this time without training the word vectors beyond their initial values from GloVe. Do you get different nearest neighbors?